## Тестирование времени обработки хэшей запрещенного ПО по БД УИКОР 

In [10]:
# Библиотеки
from datetime import datetime, timedelta
import hashlib
import random
import sys

In [63]:
# Создание хэшей
happ = set()
count = 10000
for i in range(count):
    h = hashlib.md5(str(i).encode()).hexdigest()
    happ.add(h)

In [60]:
# Время поиска без прасига
times = []
for i in range(10):
    n = random.randint(0,2*count)
    search = hashlib.md5(str(n).encode()).hexdigest()
    
    # Поиск
    start = datetime.now()
    res = search in happ
    time = datetime.now() - start
    
    print(f'{i+1}: {n} -> {res}')
    times.append(time.total_seconds())

mead = sum(times)/len(times)    
print('\nСреднее время поиска:', timedelta(seconds=mead))

1: 1223985 -> False
2: 1588082 -> False
3: 1586846 -> False
4: 740190 -> True
5: 105359 -> True
6: 1162273 -> False
7: 984072 -> True
8: 818790 -> True
9: 1252033 -> False
10: 1788139 -> False

Среднее время поиска: 0:00:00.000005


In [61]:
# Функция ZIEM поиска по полям
def check_filter(event, filtr):
    # проверка события по фильтрам
    if event['tax'] not in filtr['tax']:
        return False
    if 'exc' in filtr:
        # проверка исключающего фильтра
        for field, value in filtr['exc'].items():
            if field in event:
                if field == 'time':
                    check_field = str(event[field].hour)
                else:
                    if event[field]:
                        check_field = event[field].lower()
                if check_field in value:
                    return False
    if 'inc' in filtr:
        # проверка включающего фильтра
        for field, value in filtr['inc'].items():
            if field not in event:
                return False
            if field == 'time':
                check_field = str(event[field].hour)
            else:
                check_field = event[field].lower()
            if check_field not in value:
                return False
    if 'diff' in filtr:
        # проверка контроля значения
        if filtr['diff'] not in event:
            return False
    return 1

In [62]:
# Оценка времени поиска в ЗИЕМ
times = []

filtr = {'tax': '1', 
         'inc': {'md5': list(happ)}
        }

for i in range(10):
    n = random.randint(0,2*count)
    search = hashlib.md5(str(n).encode()).hexdigest()
    

    event = {'tax':'1', 
             'md5': search}
    
    # Поиск
    start = datetime.now()
    res = check_filter(event, filtr)
    time = datetime.now() - start
    
    print(f'{i+1}: {n} -> {res}')
    times.append(time.total_seconds())

mead = sum(times)/len(times)    
print('\nСреднее время поиска:', timedelta(seconds=mead))

1: 560443 -> 1
2: 1107001 -> False
3: 267128 -> 1
4: 1595022 -> False
5: 458882 -> 1
6: 662779 -> 1
7: 777011 -> 1
8: 1679170 -> False
9: 529538 -> 1
10: 1033775 -> False

Среднее время поиска: 0:00:00.080724
